In [1]:
import math
import openai
import os
import sklearn.metrics
import random
from tqdm import tqdm
from langchain.llms import OpenAI

In [2]:
import baseten
baseten.login("yiVOous9.2mjkPJSPdya6FUFbGKmXaLyUF6ZxTQYs")

INFO API key set.
INFO:baseten:API key set.


In [3]:
from task_utils import TASKS, load_data, load_prompt, generate_prompts

In [4]:
## your LLM stack goes here

# Falcon 7B

model = baseten.deployed_model_version_id('q049013')

def call_llm(prompt):
    output = model.predict({"prompt": prompt})
    output = output["data"]["generated_text"]
    result = output.rsplit(" ", maxsplit=1)[-1]
    return result
    

In [5]:
model.predict({"prompt": "what is life?"})

{'data': {'generated_text': 'what is life?\nLife is a complex and dynamic phenomenon that is characterized by the presence of living'}}

In [6]:
def evaluate(tasks: list, tasks_dir: str):
    report = dict()
    for task in tqdm(tasks):
        train_df, test_df = load_data(task=task, tasks_dir=tasks_dir)
        prompt_template = load_prompt(prompt_name="base_prompt.txt", task=task, tasks_dir=tasks_dir)
        prompts = generate_prompts(prompt_template=prompt_template, data_df=train_df)
        report[task] = dict()
        targets = list()
        outputs = list()
        for prompt, data in zip(prompts, train_df.iterrows()):
            datapoint_id, data = data
            output = call_llm(prompt)
            output = output.strip()
            targets.append(data['answer'])
            outputs.append(output)
            success = output == data['answer']
            report[task][datapoint_id] = {
                'prompt': prompt,
                'generated_output': output,
                'correct_output': data['answer'],
                'success': output == data['answer']
            }
        report[task]['balanced_accuracy'] = sklearn.metrics.balanced_accuracy_score(targets, outputs)
    
    print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))
    
    return report


In [7]:
len(TASKS)

162

In [8]:
tasks_dir = '../legalbench'

# tasks=random.sample(TASKS, 10)

# tasks= [task for task in TASKS if task.startswith("cuad")]

In [9]:
import json

with open("task_label_dict.json", "r") as f:
    task_label_dict = json.loads(f.read())

In [10]:
binary_yes_no_tasks = [task for task in task_label_dict.keys() if task_label_dict[task]["type"] == "binary_yes_no"]

In [11]:
len(binary_yes_no_tasks)

109

In [12]:
tasks = binary_yes_no_tasks

In [13]:
report = evaluate(tasks=tasks, tasks_dir=tasks_dir)

  4%|▎         | 4/109 [00:13<07:12,  4.12s/it]

In [ ]:
print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))

In [ ]:
report